# Fuzzy Logic Inference
** textbook question 7.5 (page 143) as example **

In [1]:
from __future__ import print_function
import numpy as np

## Given

In [3]:
A1 = np.array([0.0, 1.0, 0.6])
B = np.array([0.2, 0.8])

## Rule "If V is A1 then W is B" translates into
Using Eq7.6 of "Correlation Product" on page 130

In [4]:
Rule = np.outer(A1, B) # same as just taking outer product
print(Rule)

[[ 0.    0.  ]
 [ 0.2   0.8 ]
 [ 0.12  0.48]]


** Another e.g. **
If question was asking for "Correlation Min", instead, rule would've been translated into

In [9]:
np.array([min(i,j) for i in A1 for j in B]).reshape(3,2) # kinda like outer product but with 'min' as operator

array([[ 0. ,  0. ],
       [ 0.2,  0.8],
       [ 0.2,  0.6]])

**Side note** Surely there's a vectorized way of doing this:

In [24]:
np.minimum(A1[...,np.newaxis], B[np.newaxis,...])

array([[ 0. ,  0. ],
       [ 0.2,  0.8],
       [ 0.2,  0.6]])

**Side note** Not familiar with matlab, but I might do something like this:

In [30]:
repmat = lambda M,m,n: np.tile(M,(m,n))
_A1 = repmat(A1.reshape(3,1),1,2)
_B = repmat(B.reshape(1,2),3,1)
np.minimum(_A1, _B)

array([[ 0. ,  0. ],
       [ 0.2,  0.8],
       [ 0.2,  0.6]])

** Another e.g. ** And if according to Lukasiewicz's implication, the rule would've become

In [10]:
np.array([min(1.,1-i+j) for i in A1 for j in B]).reshape(3,2)

array([[ 1. ,  1. ],
       [ 0.2,  0.8],
       [ 0.6,  1. ]])

## If "V is A1" was also a known fact, then we'd have
Using Eq7.8 on page 131

In [21]:
""" Conclusion = [sup(min(0,0) min(1,0.2) min(0.6,0.12)) sup(min(0,0) min(1,0.8) min(0.6,0.48))]
    = [sup(0 0.2 0.12) sup(0 0.8 0.48)]
    = [0.2 0.8]
    = B
"""

print('min:',np.minimum(A1[...,np.newaxis],Rule)) # Taking elementwise minimum of A1 with each column in Rule
print('sup:',np.max(np.minimum(A1[...,np.newaxis],Rule), axis=0)) # Taking supremum on each column

min: [[ 0.    0.  ]
 [ 0.2   0.8 ]
 [ 0.12  0.48]]
sup: [ 0.2  0.8]


Recall the value of B, and one could find that the conclusion is **exactly** B (due to fact is exactly A1)

In [19]:
B

array([ 0.2,  0.8])

So this whole calculation reads "**If V is A1 then W is B** and **V is A1** as a matter of fact, so we obtain **W is B**"

## But what if "V is NOT A1"
Just take A1's complement instead:

In [20]:
np.max(np.minimum((1-A1)[...,np.newaxis],Rule), axis=0)

array([ 0.12,  0.4 ])

This concludes the revisit of question 7.5

And the general framework of fuzzy logic inference includes the following steps (page 133 and 134):

1. Cylindrical closure of antecedents (join together all conditions)
2. Compute implication relation (question 7.5)
3. Draw conclusions with compusitional rule of inference (question 7.5)
4. Conclusion aggregation (question 7.2)
5. Defuzzification

# A fuzzy control vacuum cleaner

In [31]:
# Input
dirty_level = dict(DIRTY = np.array([0., 1.]), MEDIUM = np.array([.7, .3]), CLEAN = np.array([1., 0.]))
battery_level = dict(HIGH = np.array([.1, .9]), LOW = np.array([.8, .2]))

# Output
mode = dict(CLEANING = np.array([0.0,0.0,1.0]), STANDBY = np.array([0.0,1.0,0.0]), CHARGING = np.array([1.0,0.0,0.0]))

fuzzify = lambda x: np.array([1.0-x, x])

# Rules
"""
    DIRTY and BATTERY HIGH => CLEANING
    MEDIUM and BATTERY HIGH => CLEANING
    BATTERY LOW => CHARGING
    CLEAN => STANDBY
"""

Rules = [
    [ np.outer(dirty_level['DIRTY'], mode['CLEANING']),
      np.outer(dirty_level['MEDIUM'], mode['CLEANING']),
      np.outer(dirty_level['CLEAN'], mode['STANDBY']) ],
    [ np.outer(battery_level['HIGH'], mode['STANDBY']),
      np.outer(battery_level['LOW'], mode['CHARGING']) ]
]

def control1(dirty, battery):
    d = fuzzify(dirty)
    b = fuzzify(battery)
    